# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from datetime import datetime
import os
import requests

In [ ]:
# Jautājumu saraksts
questions_data = [
    ("Ko dara pygame.init()?", [
        "Inicializē visus importētos pygame moduļus",
        "Palaiž spēles ciklu",
        "Ielādē spēles logu",
        "Izveido galveno varoņa sprite objektu"
    ], 0),
    ("Kura metode tiek izmantota, lai uzzīmētu taisnstūri pygame logā?", [
        "pygame.draw.rect()",
        "pygame.screen.rectangle()",
        "pygame.create.rect()",
        "pygame.render.rect()"
    ], 0),
    ("Kā tiek atjaunināts logs pygame pēc zīmēšanas?", [
        "pygame.display.flip()",
        "pygame.update.display()",
        "pygame.screen.refresh()",
        "pygame.render.frame()"
    ], 0),
    ("Ko pārbauda event.type == pygame.QUIT?", [
        "Vai lietotājs ir nospiedis loga aizvēršanas pogu",
        "Vai lietotājs nospieda taustiņu Enter",
        "Vai spēlei jāsākas no jauna",
        "Vai spēlētājs zaudēja"
    ], 0),
    ("Kurš modulis tiek izmantots, lai veidotu GUI lietotnes Python valodā?", [
        "tkinter",
        "pygame.gui",
        "guimodule",
        "tkbuilder"
    ], 0),
    ("Kurš tkinter logelements tiek izmantots, lai saņemtu tekstu no lietotāja?", [
        "Entry",
        "Label",
        "Button",
        "Canvas"
    ], 0),
    ("Ko dara mainloop() tkinter lietotnē?", [
        "Palaiž GUI notikumu ciklu",
        "Pieslēdzas datubāzei",
        "Atjauno Python vidi",
        "Izveido loga izvietojumu"
    ], 0),
    ("Kurš tkinter logelements ļauj zīmēt figūras kā taisnstūrus un tekstu?", [
        "Canvas",
        "Frame",
        "Box",
        "Entry"
    ], 0),
]

random.shuffle(questions_data)
results = [None] * len(questions_data)
attempts = [0] * len(questions_data)
question_widgets = []
submit_button = widgets.Button(description="Iesniegt rezultātu", button_style='success')
download_button = widgets.Button(description="⬇️ Lejupielādēt rezultātu", button_style='info')
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')

In [ ]:
# Google Forms konfigurācija
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140"
}
for i in range(1, 9):
    entry_ids[str(i)] = f"entry.200000000{i}"  # Piemērs

In [ ]:
def create_question(index, question, options, correct_index):
    out = widgets.Output()
    paired = list(enumerate(options))
    random.shuffle(paired)
    shuffled_indexes, shuffled_options = zip(*paired)
    correct_shuffled_index = shuffled_indexes.index(correct_index)

    radio = widgets.RadioButtons(options=shuffled_options)
    button = widgets.Button(description='Pārbaudīt')
    feedback = widgets.HTML()

    def on_click(b):
        attempts[index] += 1
        if radio.index == correct_shuffled_index:
            penalty = 5 * (attempts[index] - 1)
            score = max(12.5 - penalty, 0)
            results[index] = score
            feedback.value = "<span style='color:green'>✔️ Pareizi!</span>"
            radio.disabled = True
            button.disabled = True
            if all(r is not None for r in results):
                display(widgets.VBox([name_input, submit_button]))
        else:
            feedback.value = "<span style='color:red'>❌ Nepareizi. Mēģini vēlreiz.</span>"

    button.on_click(on_click)

    with out:
        display(widgets.HTML(f"<b>{index+1}. {question}</b>"), radio, button, feedback)
    return out

In [ ]:
def calculate_score(_):
    clear_output()
    for w in question_widgets:
        display(w)
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    color = 'green' if final_score >= 70 else 'red'
    status = '✅ Nokārtots!' if final_score >= 70 else '❌ Nenokārtots.'
    display(widgets.HTML(f"<h3>Rezultāts: <span style='color:{color}'>{final_score}%</span> – {status}</h3>"))
    display(download_button)

    # Nosūtīšana uz Google Form
    student_name = name_input.value.strip() or "Nezināms"
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    form_data = {
        entry_ids["Students"]: student_name,
        entry_ids["Datums"]: now,
        entry_ids["Rezultats"]: str(final_score)
    }

    for i in range(len(questions_data)):
        question_text = questions_data[i][0]
        score = results[i] if results[i] is not None else 0
        form_data[entry_ids[str(i+1)]] = f"{question_text} (Mēģinājumi: {attempts[i]}) – {score} punkti"

    try:
        requests.post(form_url, data=form_data)
    except:
        display(widgets.HTML("<b style='color:red'>Neizdevās nosūtīt uz Google Formu.</b>"))